In [1]:
import xarray as xr 
import iris
import matplotlib.pyplot as plt 
import numpy as np 

import efp_saffin as saff 
import datetime as dt 

# JRA55 Data

### Maths servers:

In [2]:
# ds = xr.open_mfdataset('/home/links/ct715/eddy_feedback/daily_datasets/jra55_djf_uvtw_ubar_ep.nc', 
#                        parallel=True, chunks={'time':31})

# ds

### JASMIN servers:

In [3]:
ds = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/reanalysis_data/jra55_daily/jra55_djf_ubar_ep.nc',
                            parallel=True, chunks={'time': 31})

ds

<xarray.Dataset>
Dimensions:  (time: 5325, lon: 144, lat: 73, level: 37)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01T09:00:00 ... 2016-12-31T09:00:00
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * level    (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    v        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    t        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    omega    (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    u        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    ep1      (time, level, lat) float64 dask.array<chunksize=(31, 37, 73), meta=np.ndarray>
    ep2      (time, level, lat) float64 dask.array<chunksize=(31, 37, 73), meta=np.ndarray>
    div1     (time, level, lat) float64 dask.array<chunksize=(31, 37, 73), meta=np.ndarray>
    div2     (time, level, lat) float64 dask.array<chunksize=(31, 37, 73), meta=np.ndarray>
    ubar     (time, level, lat) float32 dask.array<chunksize=(31, 37, 73), meta=np.ndarray>

In [4]:
# convert xarray to iris cubes
ubar = ds.ubar.to_iris()
div1 = ds.div1.to_iris()

# calculate seasonal mean (needs to be subsetted to DJF already)
span = dt.timedelta(days=28)
ubar = saff.seasonal_mean(ubar, span)
div1 = saff.seasonal_mean(div1, span)

# slice at 500hPa
ubar = ubar.extract(iris.Constraint(level=500.))
div1 = div1.extract(iris.Constraint(level=500.))

In [5]:
efp, efp_array = saff.eddy_feedback_parameter(div1, ubar)

/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2245: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'season_year', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'season_year'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2245: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate i

In [6]:
efp.data 

array(0.32333659)

In [7]:
# efp_array.data

In [8]:
# plt.figure()
# plt.plot(efp_array.coord('latitude').points, efp_array.data)
# plt.show()

## QG Eddy Feedback Parameter

In [9]:
# extract DataArray and convert to iris cubes
U = ds.u.to_iris()
V = ds.v.to_iris()

# need to set units for whatever reason...
U.units = 'm s-1'
V.units = 'm s-1'

# calculate EP flux div and set variables for EFP
divFh = saff.horiz_EPflux_div(U,V) 
Ubar = ds.ubar.to_iris() 

# take seasonal mean
divFh = saff.seasonal_mean(divFh, span)
Ubar = saff.seasonal_mean(Ubar, span) 

# take slice at 500hPa
divFh = divFh.extract(iris.Constraint(level=500))
Ubar = Ubar.extract(iris.Constraint(level=500)) 

/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'longitude', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'longitude'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'longitude', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'longitude'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'longitude', with 0 bound(s). Contiguous bounds a

In [10]:
EFP, EFP_array = saff.eddy_feedback_parameter(divFh, Ubar)

/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2245: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'season_year', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'season_year'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2245: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate i

In [11]:
EFP.data 

array(0.23673757)

In [12]:
EFP_array.data

masked_array(data=[0.19352664, 0.32079548, 0.42384592, 0.49769879,
                   0.47717825, 0.33264652, 0.26608726, 0.3089646 ,
                   0.36629642, 0.34593377, 0.23484436, 0.15862829,
                   0.05516373, 0.02383537, 0.10902585, 0.1662012 ,
                   0.20117915, 0.27200872, 0.1528502 ],
             mask=False,
       fill_value=1e+20)

In [13]:
# plt.figure()
# plt.plot(EFP_array.coord('latitude').points, EFP_array.data)
# plt.show()

In [14]:
# fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,5))

# ax1.plot(efp_array.coord('latitude').points, efp_array.data)
# ax1.set_xlabel('Lat')
# ax1.set_ylabel('Corr$^2$(div1, ubar)')
# ax1.set_title('Full primitive (calc using aostools)')

# ax2.plot(EFP_array.coord('latitude').points, EFP_array.data)
# ax2.set_xlabel('Lat')
# ax2.set_ylabel('Corr$^2$(div1, ubar)')
# ax2.set_title('QG version (calc using Saffin code)')

# plt.show()

# ERA5 Data

### Maths servers:

In [15]:
# ds = xr.open_mfdataset('/home/links/ct715/eddy_feedback/daily_datasets/era5daily_djf_uvt_ubar_ep.nc', 
#                         parallel=True, chunks={'time':31})

# ds

### JASMIN Servers:

In [16]:
ds = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/reanalysis_data/era5_daily/era5daily_djf_uvt_ubar_ep.nc',
                       parallel=True, chunks={'time':31})
ds

<xarray.Dataset>
Dimensions:  (time: 4030, lon: 360, lat: 181, level: 37)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01T09:00:00 ... 2023-02-28T09:00:00
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * lat      (lat) float32 90.0 89.0 88.0 87.0 86.0 ... -87.0 -88.0 -89.0 -90.0
  * level    (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    t        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 181, 360), meta=np.ndarray>
    u        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 181, 360), meta=np.ndarray>
    v        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 181, 360), meta=np.ndarray>
    ubar     (time, level, lat) float32 dask.array<chunksize=(31, 37, 181), meta=np.ndarray>
    ep1      (time, level, lat) float64 dask.array<chunksize=(31, 37, 181), meta=np.ndarray>
    ep2      (time, level, lat) float64 dask.array<chunksize=(31, 37, 181), meta=np.ndarray>
    div1     (time, level, lat) float64 dask.array<chunksize=(31, 37, 181), meta=np.ndarray>
    div2     (time, level, lat) float64 dask.array<chunksize=(31, 37, 181), meta=np.ndarray>

In [17]:
# convert to iris cubes
ubar = ds.ubar.to_iris()
div1 = ds.div1.to_iris()

# take seasonal mean (DataArray needs to already be subsetted to DJF)
ubar = saff.seasonal_mean(ubar, span)
div1 = saff.seasonal_mean(div1, span)

# take slice at 500hPa
ubar = ubar.extract(iris.Constraint(air_pressure=500.))
div1 = div1.extract(iris.Constraint(air_pressure=500.))

In [18]:
efp2, efp_array2 = saff.eddy_feedback_parameter(div1, ubar)

/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2245: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'season_year', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'season_year'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2245: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate i

In [19]:
efp2.data

array(0.32050844)

In [20]:
efp_array2.data

masked_array(data=[0.08996887, 0.12933676, 0.16776018, 0.23587432,
                   0.27667464, 0.32672073, 0.37382674, 0.3735124 ,
                   0.33297913, 0.33242376, 0.38054312, 0.45378884,
                   0.45976982, 0.36817931, 0.34594904, 0.2931599 ,
                   0.27780251, 0.34616045, 0.31548361, 0.36588604,
                   0.43650349, 0.41772134, 0.50492062, 0.56728369,
                   0.54859049, 0.43588911, 0.45690681, 0.52308378,
                   0.52055295, 0.48420229, 0.37817886, 0.28422337,
                   0.15334243, 0.18040857, 0.19457043, 0.09590255,
                   0.11223702, 0.24233881, 0.28486134, 0.21634461,
                   0.14528485, 0.1324223 , 0.26443272, 0.38225239,
                   0.40936161, 0.26473157, 0.2643049 , 0.32496669],
             mask=False,
       fill_value=1e+20)

In [21]:
# plt.figure()
# plt.plot(efp_array.coord('latitude').points, efp_array.data)
# plt.show()

## QG version

In [22]:
# extract dataArray and convert to iris cubes
U = ds.u.to_iris()
V = ds.v.to_iris()
# need to change units for whatever reason
U.units = 'm s-1'
V.units = 'm s-1'

# define EFP variables
divFh = saff.horiz_EPflux_div(U,V) 
Ubar = ds.ubar.to_iris() 

# take seasonal mean
divFh = saff.seasonal_mean(divFh, span)
Ubar = saff.seasonal_mean(Ubar, span) 

# slice data at 500hPA
divFh = divFh.extract(iris.Constraint(air_pressure=500))
Ubar = Ubar.extract(iris.Constraint(air_pressure=500)) 

/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'longitude', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'longitude'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'longitude', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'longitude'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'longitude', with 0 bound(s). Contiguous bounds a

: 

In [ ]:
EFP2, EFP_array2 = saff.eddy_feedback_parameter(divFh, Ubar)

In [ ]:
EFP2.data 

In [ ]:
# plt.figure()
# plt.plot(EFP_array.coord('latitude').points, EFP_array.data)
# plt.show()

In [ ]:
# fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,5))

# ax1.plot(efp_array.coord('latitude').points, efp_array.data)
# ax1.set_xlabel('Lat')
# ax1.set_ylabel('Corr$^2$(div1, ubar)')
# ax1.set_title('Full primitive (calc using aostools)')

# ax2.plot(EFP_array.coord('latitude').points, EFP_array.data)
# ax2.set_xlabel('Lat')
# ax2.set_ylabel('Corr$^2$(div1, ubar)')
# ax2.set_title('QG version (calc using Saffin code)')

# plt.show()

# Summary

In [ ]:
print(f'JRA55 full primitive: {efp.data}')
print(f'JRA55 QG version (Saffin): {EFP.data}') 
print()
print(f'ERA5 full primitive: {efp2.data}')
print(f'ERA5 QG version (Saffin): {EFP2.data}') 

JRA55 full primitive: 0.32333658579344904

ERA5 full primitive: 0.3205084452489285
